In [1]:
import numpy as np
from numpy.linalg import eig
from numpy.linalg import inv,pinv
import pandas as pd
import csv
from collections import defaultdict
from functools import partial

In [2]:
def create_dict_from_arrays(keys, values):
    keys_sorted_arg = keys.argsort()
    keys = keys[keys_sorted_arg]
    values = values[keys_sorted_arg]
    a = arr[arr[:,0].argsort()] # sort by col-0 if not already sorted
    s0 = np.r_[0,np.flatnonzero(a[1:,0] > a[:-1,0])+1,a.shape[0]]
    ids = a[s0[:-1],0]
    return {ids[i]:a[s0[i]:s0[i+1],1].tolist() for i in range(len(s0)-1)}

In [101]:
def get_x_grouped_by_class(x, y):
    y = np.asarray(y)
    x = np.asarray(x)

    unique_y = np.unique(y[:, 0])

    dataset = {key: [] for key in unique_y}
    for i in range(y.shape[0]):
        dataset[y[i][0]].append(x[i][:].tolist())
        
    return dataset

class fisher_projection:  
    def __init__(self, proj_dim):
        self.proj_dim = proj_dim
        self.W = None # weights
        self.proj = None
    
    def fit(self, x, y):
        y = np.asarray(y)
        x = np.asarray(x)

        unique_y = np.unique(y[:, 0])

        dataset = {key: [] for key in unique_y}
        for i in range(y.shape[0]):
#             print(dataset[y[i][0]])
#             print(x[i])
            dataset[y[i][0]].append(x[i][:].tolist())
        
#         print(dataset)
        s_w = np.zeros((x.shape[1], x.shape[1]))
        s_b = np.zeros((x.shape[1], x.shape[1]))
        m = np.mean(x, axis=0)
#         print("m: ", m.shape)
        for y_k, x_k in dataset.items():
            x_k = np.asarray(x_k)
            m_k = np.mean(x_k, axis=0)
#             sub = x_k.T - np.broadcast_to(m_k, (m_k.shape[0], x_k.shape[0]))
            sub = x_k.T - np.reshape(m_k, (m_k.shape[0], 1))
            s_k = np.dot(sub, np.transpose(sub))
            s_w += s_k
            
            sub = m_k - m
            s_bk = np.multiply(len(x_k), np.outer(sub, sub.T))
            s_b += s_bk
            
#         print("s_b: ", pd.DataFrame(s_b))
        sw_sb = np.dot(pinv(s_w), s_b)
        # find eigen values and eigen-vectors pairs for np.dot(pinv(SW),SB)
        eig_values, eig_vectors = eig(sw_sb)
#         print(eig_values)
#         print(eig_vectors)
        
        eig_values_sorted_arg = eig_values.argsort()[::-1]
        self.W = eig_vectors[:, eig_values_sorted_arg[: self.proj_dim]].real
#         print("w: ", pd.DataFrame(self.W))

        
        self.proj = np.dot(x, self.W)
        
#         print(self.proj.shape)
        
    def project(self, x):
        return np.dot(x, self.W)
        
        
class gaussian_discriminant:
  
    def __init__(self):
        self.g_means, self.g_covariance, self.class_priors = None, None, None
#         self.score = None
        self.N = None
    

    def fit(self, x, y):
        dataset = get_x_grouped_by_class(x, y)
        self.N = len(np.unique(y))
        means = {}
        covariance = {}
        priors = {}  # p(Ck)
        
        for class_id, features_k in dataset.items():
            features_k = np.asarray(features_k)
            means[class_id] = np.mean(features_k, axis=0)
            covariance[class_id] = np.cov(features_k, rowvar=False)
            priors[class_id] = features_k.shape[0] / self.N
            
        self.g_means = means
        self.g_covariance = covariance
        self.class_priors = priors
            
  
    # model a multi-variate Gaussian distribution for each class’ likelihood distribution P(x|Ck)
    def gaussian_distribution(self, x, u, cov):
        scalar = (1. / ((2 * np.pi) ** (x.shape[0] / 2.))) * (1 / np.sqrt(np.linalg.det(cov)))
        x_sub_u = np.subtract(x, u)
        return scalar * np.exp(-np.dot(np.dot(x_sub_u, inv(cov)), x_sub_u.T) / 2.)
    

    def score(self, x, y):
        gaussian_likelihoods = []
        classes = np.asarray(list(self.g_means.keys()))
        for x_i in x:
            row = []
            for class_id in classes:  # number of classes
                gaussian_dist = self.gaussian_distribution(x_i, self.g_means[class_id], self.g_covariance[class_id]) 
                res = self.class_priors[class_id] * gaussian_dist
                # Compute the posterios P(Ck|x) prob of a class k given a point x
                row.append(res)
#             print(row)
#             exit()
            gaussian_likelihoods.append(row)

        gaussian_likelihoods = np.asarray(gaussian_likelihoods)

        # assign x to the class with the largest posterior probability
        predictions = classes[np.argmax(gaussian_likelihoods, axis=1)]
        return np.sum(predictions == y[:, 0]) / len(y), predictions
    

def get_k_fold_indices(k, n, shuffle=True):
    
    indices = np.arange(n)
    if shuffle == True:
        np.random.shuffle(indices)
        
    k_folded_indices = np.array_split(indices, k)
    k_folded_train_indices = []
    k_folded_test_indices = []
    
    for i in range(k):
        test_indices = k_folded_indices[i]
        train_indices = np.setdiff1d(indices, test_indices, )
        k_folded_train_indices.append(train_indices)
        k_folded_test_indices.append(k_folded_indices[i])
        
#         print(k_folded_indices[:4])
#     print(k_folded_test_indices)
    
    return k_folded_train_indices, k_folded_test_indices

In [102]:
with open('digits.csv', 'r') as csvfile:
    digitDataset = np.asarray(list(csv.reader(csvfile, quoting=csv.QUOTE_NONNUMERIC)))
    
print(digitDataset.shape)
x = digitDataset[:, :-1]
y = digitDataset[:, -1:]

fisher_proj = fisher_projection(2)

k_folded_train_indices, k_folded_test_indices = get_k_fold_indices(10, y.shape[0], shuffle=True)
folds_before_fisher_errors = []
folds_after_fisher_errors = []


for fold, train_indices in enumerate(k_folded_train_indices):
    test_indices = k_folded_test_indices[fold]
    
    train_x = x[train_indices]
    test_x = x[test_indices]
    train_y = y[train_indices]
    test_y = y[test_indices]
    
    fisher_proj.fit(train_x, train_y)
    projected_train_features = fisher_proj.project(train_x)

    clf = gaussian_discriminant()
    clf.fit(projected_train_features, train_y)

    projected_test_features = fisher_proj.project(test_x)
    accuracy, predictions = clf.score(projected_test_features, test_y)
    folds_before_fisher_errors.append(1 - accuracy)


fisher_proj.fit(x, y)

for fold, train_indices in enumerate(k_folded_train_indices):
    test_indices = k_folded_test_indices[fold]
    
    train_x = x[train_indices]
    test_x = x[test_indices]
    train_y = y[train_indices]
    test_y = y[test_indices]
    

    projected_train_features = fisher_proj.project(train_x)
    
    clf = gaussian_discriminant()
    clf.fit(projected_train_features, train_y)

    projected_test_features = fisher_proj.project(test_x)
    accuracy, predictions = clf.score(projected_test_features, test_y)
    folds_after_fisher_errors.append(1 - accuracy)

print(folds_before_fisher_errors, np.mean(folds_before_fisher_errors))
print(folds_after_fisher_errors, np.mean(folds_after_fisher_errors))

(1797, 65)
[0.36111111111111116, 0.27222222222222225, 0.3277777777777777, 0.3388888888888889, 0.24444444444444446, 0.30000000000000004, 0.33333333333333337, 0.2960893854748603, 0.3016759776536313, 0.36871508379888274] 0.31442582247051526
[0.33333333333333337, 0.2833333333333333, 0.34444444444444444, 0.2833333333333333, 0.26111111111111107, 0.28888888888888886, 0.2666666666666667, 0.2849162011173184, 0.26815642458100564, 0.3016759776536313] 0.29158597144630666
